# 5. Передача функций, как аргументов и их возвращение, как результатов. Каррирование (currying), сечение инфиксных операторов. Функции map, foldr, foldl и filter, примеры их использования. Бета- и эта-редукции, бесточечная запись.

## 5.1. Сечение инфиксных операторов.

Частичное  применение  операторов  называется  **сечением**.

Если какая-нибудь обычная функция  `f`  принимает параметры  `x`  `y`  и  `z`, то ей можно передать только  `x`, или передать только  `x` и  `y`, или передать все параметры сразу  –  но нельзя, например, передать только `y`.

А  вот  сечениями  все  проще:  `(^2)`  есть  функция,  возводящая  в  квадрат,  а  `(2^)`  есть  функция, вычисляющая  2  в заданной степени, и отличаются эти функции тем, какой именно из параметров функции возведения в степень  `(^) ::  => Double  -> Integer  -> Double`  мы передаем: первый или второй.

In [14]:
toSquare x = (^) x 2
twoToDeg x = (^) 2 x

Line 2: Eta reduce
Found:
twoToDeg x = (^) 2 x
Why not:
twoToDeg = (^) 2

In [15]:
toSquare 3

9

In [16]:
twoToDeg 3

8

Рассмотрим функцию `(+)`:

`(+) :: Num a => a -> a -> a`

Что  мы  получаем,  передавая  функции  (+)  только  один  аргумент?

Функцию  от  оставшегося  аргумента.  Это  означает,  что  тип  функции  `(+)`  можно  рассматривать  не  только  так,  как  мы 
написали, но и по-другому:

`(+) :: Num a => a -> (a -> a)`

В данном случае мы явно выделяем тот факт, что  `(+)`  на самом деле является функцией одного аргумента!  Просто  вызывая  `(+)`  с  двумя  параметрами  мы  не  успеваем  заметить  ту  стадию вычислений, на которой появляется функция одного аргумента. Но на самом деле она есть:

`(+) 2 3 -> (2+) 3 -> (2+3) -> 5`

## 5.2. Каррирование.

На любую функцию вида:

`f :: a  -> b  ->  с  ->  d  ->  …  ->  y  ->  z`

можно посмотреть, как на функцию:

`f :: a -> (b -> (с -> (d -> … -> (y -> z)…)`

Это означает,  что  абсолютно  все  функции  в  Haskell  имеют  ровно  один  параметр  (и  один 
результат, который может быть чем  угодно  –  в том числе и опять функцией). 

Такие функции (первый вариант) называются  **каррированными**.

В случае каррированной функции в Haskell мы обязаны писать `f x y`, и не можем писать `f (x,y)`, потому что функция принимает параметр `x`, а не кортеж `(x,y)`.

Для каррирования в Haskell есть специальные функции:

```haskell
curry :: ((a,b) -> c) -> (a -> b -> c)
curry f x y = f (x,y)
```

```haskell
uncurry :: (a -> b -> c) -> ((a,b) -> c)
uncurry f p = f (fst p) (snd p)
```

In [17]:
f x y = (x + 47) * (y + 11) + 13

In [18]:
f 11 17

1637

In [19]:
f' = uncurry f

In [20]:
f' (11, 17)

1637

## 5.3. Функции высших порядков.

### 5.3.1. Map.

Функция возвращает список, созданный путем применения функции (первый аргумент) ко всем элементам списка, переданным в качестве второго аргумента.

Возможное определение:

```haskell
map :: (a -> b) -> [a] -> [b]
map f [] = []
map f (x:xs) = (f x) : map f xs
```

In [21]:
list = [-1, -3, 4, -12]

map abs list

[1,3,4,12]

### 5.3.2. Foldr.

Функция берет текущее значение аккумулятора (второй аргумент) и последний элемент списка (третий аргумент) и применяет  к этим значениям переданную функцию (первый аргумент), затем он берет предпоследний элемент списка и новое значение аккумулятора и так далее.

Если  выписать  все  элементы  списка  и  посмотреть,  как  к  ним  применяется  эта  операция,  то окажется, что результат формируется из вычисления вот такого выражения:

$$x_1 f  (x_2 f (x_3 f … (x_N f z)$$

На  каждом  шаге  работы  функции  бинарная  функция  f  применяется  к  очередному  элементу  и результату  вызова  той  же  самой  функции  от  хвоста  списка.  Математики  такую  операцию называют сверткой (причем **правой** сверткой). 

```haskell
foldr :: (b -> a -> a) -> a -> [b] -> a
foldr f z [] = z
foldr f z (x:xs) = f x (foldr f z xs)
```

In [22]:
startValue = 5
list = [-1, -3, 4, -12]

foldr (+) startValue list

-7

Промежуточные результаты можно узнать с помощью функции `scanr`:

In [23]:
scanr (+) startValue list

[-7,-6,-3,-7,5]

### 5.3.3. Foldl.

Функция берет переданное начальное значение (второй аргумент) и первый элемент списка (третий аргумент) и применяет к ним функцию (первый аргумент), затем вычисляет функцию с этим результатом и следующим элементом списка и так далее.

`foldl`  тоже  идет  по  списку,  как  и  `map`,  но  в отличие  от  `map`,  `foldl`  имеет  **аккумулятор**,  то  есть  память.

$$([] f x_1) f x_2) f x_3) f … f x_N)…)$$

```haskell
foldl :: (a -> b -> a) -> a -> [b] -> a
foldl f z [] = z
foldl f z (x:xs) = foldl f (f z x) xs
```

In [24]:
startValue = 64
list = [4,2,4]

foldl (/) startValue list

2.0

Промежуточные результаты можно узнать с помощью функции `scanl`:

In [25]:
scanl (/) startValue list

[64.0,16.0,8.0,2.0]

### 5.3.4. Filter.

Функция возвращает список, составленный из элементов списка (второй аргумент), удовлетворяющих условию, которое задано в виде унарного предиката первым аргументом.

```haskell
filter f [] = []
filter f (x:xs) | f x   = x : filter f xs
                | otherwise = filter f xs
```

In [26]:
list = [1,2,3,4,5,6,7,8]

filter (>5) list

[6,7,8]

## 5.4. Бесточечная запись.

В общем случае вместо:
```haskell
f x = expression x
```

достаточно написать:
```haskell
f = expression
```

## 5.5. Редукции.

$\beta$-редукция: терм `(\x -> M) N` редуцируется (переписывается) к результату подстановки `N` вместо `x` в `M`. Это подстановка фактического параметра вместо формального в тело функции при ее вызове.

$\eta$-редукция: `\x -> M x` редуцируется к `M`, если `x` не входит свободно в `M`. Действительно, эти два терма ведут себя одинаково, когда выступают в роли функции: `((\x -> M x) N)` $\beta$-редуцируется к результату подстановки `N` вместо `x` в `M x`, то есть в `M N` (здесь используется то, что `x` не входит свободно в `M`).
